In [27]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder # para realizar el Label Encoding 
from sklearn.preprocessing import OneHotEncoder  # para realizar el One-Hot Encoding
from sklearn.preprocessing import OrdinalEncoder

import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

En el pair programming de hoy usaremos el set de datos que guardastéis en el pair programming de normalización y estandarización.

In [28]:
df=pd.read_csv('../datos/salaries_modelo.csv', index_col=0)
df.head(2) #Hacemos una vista previa de nuestro DataFrame

,salary_box,experience_level,employee_residence
0,4631.904560,Entry-Level,US
1,7311.922975,Mid-Level,US


Vuestro set de datos debería tener al menos una variable categórica, el objetivo del pair programming de hoy:

- Hacer una códificación de la/las variables categóricas que tengáis en vuestro set de datos.

Empezamos con la clasificación de nuestras variable categóricas según sean nominales, ordinarias o binarias:

    -  Ordinarias: experience_level
    -  Nominales: company_location

Hacemos el encoding para variable categóricas ordinarias:

In [29]:
# Hacemos una funcion de mapeo

def encoder_map(df, columna, orden_valores):
    
    # nos creamos un diccionario vacío para hacer el map después. 
    ordinal_dict = {}
    
    # iteramos por nuestra lista de valores usando el enumerate que recordamos nos devolvía también el índice o posición de cada elemento
    for i, valor in enumerate(orden_valores):
        ordinal_dict[valor]=i+1 # le sumamos uno a la posición para no tener valores de 0.
        columna_nueva =columna + "map_funcion"

    # aplicamos el map
    df[columna_nueva] = df[columna].map(ordinal_dict)
    return df
    

In [30]:
## Experience_level:
df['experience_level'].unique()

array(['Entry-Level', 'Mid-Level', 'Senior', 'Executive Level'],
      dtype=object)

In [31]:
df = encoder_map(df, 'experience_level', ['Entry-Level', 'Mid-Level', 'Senior', 'Executive Level'])
df.head(2)

,salary_box,experience_level,employee_residence,experience_levelmap_funcion
0,4631.904560,Entry-Level,US,1
1,7311.922975,Mid-Level,US,2


#### Hacemos el endcoding para las variables categóricas nominales

Pruebo los dos métodos para decidir con cual nos quedamos

In [32]:
df_one=df.copy()
df_dummies=df.copy()

#### One-Hot Encoding

In [33]:
# Creamos una función que nos haga el método onehotencoder
def one_hot_encoder_one(df,columna,keep_first=True):
    
    # iniciamos el método de OneHot Encoder
    oh = OneHotEncoder()
    
    # hacemos la codificación de los datos para la variable dada 
    transformados = oh.fit_transform(df[[columna]])
    
    # convertimos nuestro array con la codificación hecha en un dataframe
    oh_df = pd.DataFrame(transformados.toarray())
    
    # el método get_feature_names nos va a dar el nombre de las columnas nuevas que se nos generarán
    oh_df.columns = oh.get_feature_names()
    
    # concatenamos el dataframe original con el dataframe que acabamos de crear
    final = pd.concat([df,oh_df],axis=1)
    
    # eliminamos la columna original 
    final.drop(columna, axis = 1,  inplace = True)
    return final

In [34]:
# aplicamos nuestra función al dataset para que nos haga el encoding sobre la columna employee_residence
df_one_enco = one_hot_encoder_one(df_one, "employee_residence")
df_one_enco.head()

,salary_box,experience_level,experience_levelmap_funcion,x0_CA,x0_ES,x0_FR,x0_GB,x0_IN,x0_US
0,4631.904560,Entry-Level,1,0.0,0.0,0.0,0.0,0.0,1.0
1,7311.922975,Mid-Level,2,0.0,0.0,0.0,0.0,0.0,1.0
2,3622.101327,Entry-Level,1,0.0,0.0,1.0,0.0,0.0,0.0
3,3048.666850,Mid-Level,2,0.0,0.0,0.0,0.0,1.0,0.0
4,3017.697599,Entry-Level,1,0.0,0.0,1.0,0.0,0.0,0.0


In [35]:
df_one_enco.drop("experience_level", axis=1, inplace=True)

#### get_dummies

In [36]:
# en este caso trabajaremos con la columna de owners, veamos cuales son sus valores únicos
df_one.employee_residence.unique()

array(['US', 'FR', 'IN', 'GB', 'ES', 'CA'], dtype=object)

In [37]:
# lo haremos para la columna "employee_residence"

dummies = pd.get_dummies(df_dummies["employee_residence"], prefix_sep = "_", prefix = "emp_resid", dtype = int)
dummies.head(2)

,emp_resid_CA,emp_resid_ES,emp_resid_FR,emp_resid_GB,emp_resid_IN,emp_resid_US
0,0,0,0,0,0,1
1,0,0,0,0,0,1


In [38]:
# juntamos los datos condf_dummies

df_dummies_final = pd.concat([df_dummies, dummies], axis = 1)
df_dummies_final.head(2)

,salary_box,experience_level,employee_residence,experience_levelmap_funcion,emp_resid_CA,emp_resid_ES,emp_resid_FR,emp_resid_GB,emp_resid_IN,emp_resid_US
0,4631.904560,Entry-Level,US,1,0,0,0,0,0,1
1,7311.922975,Mid-Level,US,2,0,0,0,0,0,1


In [39]:
df_dummies_final.drop(["experience_level","employee_residence"], axis=1, inplace=True)
df_dummies_final.head(2)


,salary_box,experience_levelmap_funcion,emp_resid_CA,emp_resid_ES,emp_resid_FR,emp_resid_GB,emp_resid_IN,emp_resid_US
0,4631.904560,1,0,0,0,0,0,1
1,7311.922975,2,0,0,0,0,0,1


- Guardad el dataframe, donde deberíais tener las variables estadandarizas, normalizadas y codificadas en un csv para usarlo en el próximo pairprogramming

In [40]:
df_dummies_final.to_csv('../datos/salarios_encoding_dum.csv')

In [41]:
df_one_enco.to_csv('../datos/salarios_encoding_one.csv')